[View in Colaboratory](https://colab.research.google.com/github/papasanimohansrinivas/papasanimohansrinivas.github.io/blob/master/Copy_of_DNST_CIFAR10_AUG.ipynb)

In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 6
l = 12
num_filter = 17
compression = 1.0
dropout_rate = 0.2

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [0]:
# Dense Block
def add_denseblock(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 12, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 17
dropout_rate = 0.2
l = 14
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [63]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_7 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_329 (Conv2D)             (None, 32, 32, 17)   459         input_7[0][0]                    
__________________________________________________________________________________________________
batch_normalization_329 (BatchN (None, 32, 32, 17)   68          conv2d_329[0][0]                 
__________________________________________________________________________________________________
activation_329 (Activation)     (None, 32, 32, 17)   0           batch_normalization_329[0][0]    
__________________________________________________________________________________________________
conv2d_330

conv2d_385 (Conv2D)             (None, 4, 4, 17)     28611       activation_384[0][0]             
__________________________________________________________________________________________________
dropout_378 (Dropout)           (None, 4, 4, 17)     0           conv2d_385[0][0]                 
__________________________________________________________________________________________________
concatenate_357 (Concatenate)   (None, 4, 4, 204)    0           concatenate_356[0][0]            
                                                                 dropout_378[0][0]                
__________________________________________________________________________________________________
batch_normalization_385 (BatchN (None, 4, 4, 204)    816         concatenate_357[0][0]            
__________________________________________________________________________________________________
activation_385 (Activation)     (None, 4, 4, 204)    0           batch_normalization_385[0][0]    
__________

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [65]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/6
50000/50000 [==============================] - 410s 8ms/step - loss: 1.5003 - acc: 0.4495 - val_loss: 1.3987 - val_acc: 0.5321
Epoch 2/6
19456/50000 [==========>...................] - ETA: 3:39 - loss: 1.1039 - acc: 0.5986

50000/50000 [==============================] - 383s 8ms/step - loss: 1.0507 - acc: 0.6218 - val_loss: 1.5184 - val_acc: 0.5693
Epoch 3/6
45824/50000 [==========================>...] - ETA: 30s - loss: 0.8719 - acc: 0.6895

50000/50000 [==============================] - 384s 8ms/step - loss: 0.8690 - acc: 0.6906 - val_loss: 1.2729 - val_acc: 0.6306
Epoch 4/6
50000/50000 [==============================] - 384s 8ms/step - loss: 0.7554 - acc: 0.7310 - val_loss: 0.9831 - val_acc: 0.6810
Epoch 5/6
 3456/50000 [=>............................] - ETA: 5:34 - loss: 0.7547 - acc: 0.7358

50000/50000 [==============================] - 382s 8ms/step - loss: 0.6748 - acc: 0.7613 - val_loss: 1.1138 - val_acc: 0.6768
Epoch 6/6
39680/50000 [======================>.......] - ETA: 1:13 - loss: 0.6111 - acc: 0.7870

50000/50000 [==============================] - 379s 8ms/step - loss: 0.6129 - acc: 0.7858 - val_loss: 1.1690 - val_acc: 0.6706


In [66]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 32s 3ms/step
Test loss: 1.1690057567596435
Test accuracy: 0.6706


In [54]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')

In [56]:
ls -lah


total 1.7M
drwxr-xr-x 1 root root 4.0K May  6 15:33 ./
drwxr-xr-x 1 root root 4.0K May  6 11:15 ../
drwx------ 4 root root 4.0K May  6 11:19 .cache/
drwxr-xr-x 3 root root 4.0K May  6 11:19 .config/
drwxr-xr-x 1 root root 4.0K Apr 30 16:29 datalab/
-rw-r--r-- 1 root root 1.6M May  6 16:35 DNST_model.h5
drwxr-xr-x 4 root root 4.0K May  6 11:15 .forever/
drwxr-xr-x 5 root root 4.0K May  6 11:19 .ipython/
drwxr-xr-x 3 root root 4.0K May  6 11:20 .keras/
drwx------ 3 root root 4.0K May  6 11:15 .local/
drwx------ 3 root root 4.0K May  6 11:19 .nv/
-rw------- 1 root root 1.0K May  6 11:15 .rnd
